# Webページをスクレイピング

データ分析ではCSVのようなファイルではなく、データソースとしてWebページを活用することもできます。  
Webページからデータを取得することを、 **スクレイピング** と呼びます。

本Notebookでは[CodeZine](https://codezine.jp)から`機械学習`タグの付いている記事一覧の取得を目標とします。

In [1]:
!pip install requests beautifulsoup4

In [2]:
!pip list | grep -e 'requests' -e 'beautifulsoup4' | awk '{print $1, "=", $2}'

beautifulsoup4 = 4.7.1
requests = 2.21.0


## Webページを取得

In [3]:
import requests

r = requests.get('https://codezine.jp/')
print(type(r))
print(r.status_code)

<class 'requests.models.Response'>
200


In [4]:
# <title>タグと<h1>タグの要素を取得
text = r.text
for line in text.split('\n'):
    if '<title>' in line or '<h1>' in line:
        print(line.strip())

<title>CodeZine（コードジン）</title>
<h1><a href="/"><img src="//cz-cdn.shoeisha.jp/lib/img/cmn/cmn-header-logo.png" alt="CodeZine（コードジン）" ></a></h1>


## Webページから要素を抜き出す

In [5]:
from bs4 import BeautifulSoup

# HTMLを解析
soup = BeautifulSoup(text, 'html.parser')
print(soup.title)
print(soup.h1)
print(soup.h1.a.img)  # ネストされた要素も取得できる！！

<title>CodeZine（コードジン）</title>
<h1><a href="/"><img alt="CodeZine（コードジン）" src="//cz-cdn.shoeisha.jp/lib/img/cmn/cmn-header-logo.png"/></a></h1>
<img alt="CodeZine（コードジン）" src="//cz-cdn.shoeisha.jp/lib/img/cmn/cmn-header-logo.png"/>


In [6]:
# 合致するタグを全て取り出す
atags = soup.find_all('a')
print('amount of <a> tags: ', len(atags))
for atag in atags[:5]:
    print('Title:', atag.text)
    print('Link:', atag['href'])

amount of <a> tags:  173
Title: このページの本文へ移動
Link: #contents
Title: 企業IT
Link: https://enterprisezine.jp/
Title: 開発
Link: https://codezine.jp/
Title: データベース
Link: https://enterprisezine.jp/dbonline/
Title: セキュリティ
Link: https://enterprisezine.jp/securityonline/


## 記事一覧を抜き出す

2019/08/13時点でのHTML要素の構成

* 記事一覧
    * `<ul class='catList'>`配下の`<li>`要素に各記事が格納

* 記事日付
    * `<div class='day'>`に格納

* 記事タイトルとリンク
    * `<h2>`配下の`<a href='~'>`に格納

* タグ
    * `<ul>`配下の`<li class='tag'>`に格納


In [7]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup

r = requests.get('https://codezine.jp/article/t/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92')
soup = BeautifulSoup(r.text, 'html.parser')

articles = []

# 要素を抜き出す
lis = soup.select('ul.catList > li')
for li in lis:
    # 配信日取得
    day = li.find('div', class_='day').text.strip()
    published = datetime.strptime(day, '%Y/%m/%d')

    # タイトル・URL取得
    h2_tag = li.find('h2')
    title = h2_tag.text
    url = h2_tag.a['href']

    # タグの一覧を取得
    tag_list = li.select('ul.tag > li')
    tags = [tag.text.strip() for tag in tag_list]

    # 記事をリストに追加
    article = {
        'published': published,
        'title': title,
        'url': url,
        'tags': tags
    }
    articles.append(article)

In [8]:
# 記事の先頭3件を取得
articles[:3]

[{'published': datetime.datetime(2019, 5, 31, 0, 0),
  'title': '業務で機械学習を使うためには、まず課題を洗い出す～『現場で使える！Python機械学習入門』より',
  'url': '/article/detail/11532',
  'tags': ['Python', '機械学習']},
 {'published': datetime.datetime(2019, 4, 24, 0, 0),
  'title': '機械学習アルゴリズムのしくみを解説\u3000「サポートベクトルマシン」と「k-means法」とは',
  'url': '/article/detail/11466',
  'tags': ['機械学習', 'アルゴリズム']},
 {'published': datetime.datetime(2018, 12, 21, 0, 0),
  'title': '「中の人」が教える！ OSSを“Apache”ブランドに育てるApache Incubatorのしくみとは？',
  'url': '/article/detail/11292',
  'tags': ['インタビュー', '機械学習', 'ライブラリ']}]

In [9]:
# 記事一覧は後の分析で利用しやすいpandas DataFrame型に変換
import pandas as pd
df = pd.DataFrame(articles)
df

,published,tags,title,url
0,2019-05-31,"[Python, 機械学習]",業務で機械学習を使うためには、まず課題を洗い出す～『現場で使える！Python機械学習入門』より,/article/detail/11532
1,2019-04-24,"[機械学習, アルゴリズム]",機械学習アルゴリズムのしくみを解説 「サポートベクトルマシン」と「k-means法」とは,/article/detail/11466
2,2018-12-21,"[インタビュー, 機械学習, ライブラリ]",「中の人」が教える！ OSSを“Apache”ブランドに育てるApache Incubato...,/article/detail/11292
3,2018-12-17,[機械学習],話題の「ランキング学習」とは？ 回帰法・分類法との違いからモデル構築まで,/article/detail/11283
4,2018-11-30,"[開発／設計／テスト, 機械学習]",要件定義書がしっかり読まれているかを測定――視線情報を機械学習にかけ、要件定義書レビューの評...,/article/detail/11218
5,2018-10-29,"[Python, 機械学習]",なぜいま機械学習が注目されているのか――教師あり学習、教師なし学習、強化学習の基本を理解する,/article/detail/11130
6,2018-10-24,[機械学習],AI開発を仕事にするための具体的な行動計画を『機械学習エンジニアになりたい人のための本』から紹介,/article/detail/11136
7,2018-09-25,[機械学習],数式で説明された機械学習の本を深く理解するために――大学数学の基礎を解説したシリーズ3編,/article/detail/11069
8,2018-09-04,[機械学習],機械学習エンジニアとして数学を理解しておきたい！ベクトルや行列を扱う線形代数学を学び直すために,/article/detail/11023
9,2018-07-24,"[機械学習, 人工知能]",アルファ碁ゼロに使われているディープラーニングを解き明かす 論文から詳細を紹介,/article/detail/10952
